In [4]:
import subprocess


[https://docs.python.org/zh-cn/3.8/library/subprocess.html](https://docs.python.org/zh-cn/3.8/library/subprocess.html)

推荐的调用子进程的方式是在任何它支持的用例中使用 run() 函数。对于更进阶的用例，也可以使用底层的 Popen 接口。

参考：

[https://blog.csdn.net/qdPython/article/details/131457509](https://blog.csdn.net/qdPython/article/details/131457509)

[https://www.cnblogs.com/iamjianghao/p/11896564.html](https://www.cnblogs.com/iamjianghao/p/11896564.html)

In [20]:
"""
1.列出当前目录中的文件
2.shell为 True，将通过操作系统的 shell 执行指定的命令
3.如果 capture_output 设为 true，stdout 和 stderr 将会被捕获
4.默认情况下, 文件对象是以二进制模式打开的。
  如果 encoding 或者 error 被指定, 或者 text 被设为 True,
  标准输入, 标准输出和标准错误的文件对象将通过指定的 encoding 和 errors 以文本模式打开
"""
cp = subprocess.run(["python", "test_subprocess.py"], shell=True, capture_output=True, text=True)
print(cp.args)
# 子进程的退出状态码. 通常来说, 一个为 0 的退出码表示进程运行正常.
print(cp.returncode)
# 如果 run() 是设置了 encoding, errors 或者 text=True 来运行，那么从子进程捕获到的标准输出.
print(cp.stdout)

['python', 'test_subprocess.py']
0
这是不输入参数的输出内容



In [21]:
# stdout 和 stderr 参数不应当与 capture_output 同时提供。
# 如果你希望捕获并将两个流合并在一起，使用 stdout=PIPE 和 stderr=STDOUT 来代替 capture_output
cp = subprocess.run(["python", "test_subprocess.py"], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                    text=True)
print(cp.stdout)
print(cp.stderr)

这是不输入参数的输出内容




In [13]:
# 如果 check 设为 True, 并且进程以非零状态码退出, 一个 CalledProcessError 异常将被抛出
cp = subprocess.run(["exit 1"], shell=True, check=True)
print(cp.returncode)

CalledProcessError: Command '['exit 1']' returned non-zero exit status 1.

In [22]:
# input 参数将被传递给 Popen.communicate() 以及子进程的 stdin。 如果使用此参数，它必须是一个字节序列。
cp = subprocess.run(["python", "./test_subprocess.py"], stdout=subprocess.PIPE, input="this is a string",
                    encoding="utf8")
print(cp.stdout)


this is a string



## Popen

In [14]:
# args 应当是一个程序参数的序列或者是一个单独的字符串或 path-like object
# 如果 args 是序列，则要运行的程序为 args 中的第一项
r1 = subprocess.Popen(["python", "./test_subprocess.py"], stdout=subprocess.PIPE, text=True)
print(r1.stdout.read())

r2 = subprocess.Popen("python ./test_subprocess.py", stdout=subprocess.PIPE, text=True)
print(r2.stdout.read())

这是不输入参数的输出内容

这是不输入参数的输出内容



In [13]:
# shlex.split() 可以演示如何确定 args 适当的拆分形式
import shlex

command_line = input()  # python ./test_subprocess.py -n tom
args = shlex.split(command_line)
# 由 shell 中的空格分隔的选项（例如 -n）和参数（例如 tom ）位于分开的列表元素中
print(args)
r = subprocess.Popen(args, stdout=subprocess.PIPE, text=True)
print(r.stdout.read())
# 而在需要时使用引号或反斜杠转义的参数在 shell 是单独的列表元素。

['python', './test_subprocess.py', '-n', 'tom']
这是不输入参数的输出内容
tom



In [15]:
# executable 参数指定一个要执行的替换程序
# 比如由 /bin/sh 替换为 /bin/bash

FileNotFoundError: [WinError 3] 系统找不到指定的路径。

In [5]:
# 如果 cwd 不为 None，此函数在执行子进程前会将当前工作目录改为 cwd
r2 = subprocess.Popen("python ./test_subprocess.py", cwd="F:\\a", stdout=subprocess.PIPE, text=True)
print(r2.stdout.read())

F:/a 目录下的输出内容



In [8]:
# Popen 对象支持通过 with 语句作为上下文管理器，在退出时关闭文件描述符并等待进程:
with subprocess.Popen(["python", "./test_subprocess.py"], stdout=subprocess.PIPE, text=True) as proc:
    print(proc.stdout.read())

这是不输入参数的输出内容



In [18]:
# 与进程交互：将数据发送到 stdin。 从 stdout 和 stderr 读取数据，直到抵达文件结尾。
# 返回一个 (stdout_data, stderr_data) 元组
r = subprocess.Popen("python ./test_subprocess.py", stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True)
outs, errs = r.communicate(input="communicate方法", timeout=1)
print(outs)

communicate方法



In [27]:
r1 = subprocess.Popen(["dir", "."], stdout=subprocess.PIPE, shell=True)
r2 = subprocess.Popen("python ./test_subprocess.py", stdin=r1.stdout, stdout=subprocess.PIPE, text=True)
r1.stdout.close()
out = r2.communicate()[0]
print(out)

r = subprocess.Popen("dir . | python ./test_subprocess.py", shell=True, stdout=subprocess.PIPE, text=True)
out = r.communicate()[0]
print(out)

 驱动器 H 中的卷是 张国光U盘
 卷的序列号是 3D33-F24F

 H:\python\code\pythoncode\basic\website\Python 标准库\subprocess 的目录

2023/11/02  16:36    <DIR>          .
2023/11/02  16:36    <DIR>          ..
2023/11/14  10:39            16,976 tutorial.ipynb
2023/11/14  10:36               344 test_subprocess.py
               2 个文件         17,320 字节
               2 个目录 13,161,119,744 可用字节


 驱动器 H 中的卷是 张国光U盘
 卷的序列号是 3D33-F24F

 H:\python\code\pythoncode\basic\website\Python 标准库\subprocess 的目录

2023/11/02  16:36    <DIR>          .
2023/11/02  16:36    <DIR>          ..
2023/11/14  10:39            16,976 tutorial.ipynb
2023/11/14  10:36               344 test_subprocess.py
               2 个文件         17,320 字节
               2 个目录 13,161,119,744 可用字节


